In [ ]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 8)
!date

%load_ext autoreload
%autoreload 2

# Load results from a distributed run of the uk_econ sim

In [ ]:
import glob
flist = list(sorted(glob.glob('/share/costeffectiveness/results/test/*/output.hdf')))
fname = flist[-1] # most recent run with output data
print('loading from', fname)
df = pd.read_hdf(fname)
df

In [ ]:
stratification_cols = ['taxes.health_benefit', 'taxes.non_health_fraction']
df['deaths'] = df.death_count.astype(float)
df['utility'] = df.utility.astype(float)
df['taxes_spent_on_health'] = df.taxes_spent_on_health.astype(float)
df['gdp_pc'] = df.gdp_pc.astype(float)

In [ ]:
print('Deaths')
t = df.groupby(stratification_cols).deaths.describe()
np.round(t, 1)['mean'].unstack()

In [ ]:
print('Total utility (thousand utils)')
t = df.groupby(stratification_cols).utility.describe()
np.round(t/1e3, 1)['mean'].unstack()

In [ ]:
print('Total health spending (millions)')
t = df.groupby(stratification_cols).taxes_spent_on_health.describe()
np.round(t/1e6, 1)['mean'].unstack()

In [ ]:
baseline = df[(df['taxes.health_benefit'] == 0) & (df['taxes.non_health_fraction'] == 0.9)]
baseline

In [ ]:
df2 = pd.merge(df, baseline.filter(['deaths', 'utility', 'taxes_spent_on_health']),
                 left_index=True, right_index=True, how='left',
                 suffixes=('', '_baseline'))
df2['deaths_averted'] = df2.deaths_baseline - df2.deaths
df2['increase_in_utility'] = df2.utility - df2.utility_baseline
df2['cost'] = df2.taxes_spent_on_health - df2.taxes_spent_on_health_baseline

df2['utility_per_death_averted'] = df2.increase_in_utility / df2.deaths_averted
df2['cost_per_death_averted'] = df2.cost / df2.deaths_averted

In [ ]:
t = df2.groupby(stratification_cols).deaths_averted.describe()
print('Deaths Averted')
np.round(t, 1)['mean'].unstack()

In [ ]:
t = df2.groupby(stratification_cols).increase_in_utility.describe()
print('Increase in total utility')
np.round(t, 1)['mean'].unstack()

In [ ]:
t = df2.groupby(stratification_cols).cost.describe()
print('Increase in total health spending (millions)')
np.round(t/1e6, 1)['mean'].unstack()

In [ ]:
t = df2.groupby(stratification_cols).utility_per_death_averted.describe()
print('Increase in utility per death averted')
np.round(t, 1)['mean'].unstack().iloc[1:]

In [ ]:
t = df2.groupby(stratification_cols).cost_per_death_averted.describe()
print('Increase in total health spending per death averted (millions)')
np.round(t/1e3, 1)['mean'].unstack().iloc[1:]

In [ ]:
t = df2.groupby(stratification_cols).gdp_pc.describe()
print('GDP per capita (thousands)')
np.round(t/1e3, 1)['mean'].unstack().iloc[1:]

In [ ]:
np.exp(5)

In [ ]:
t = df2.groupby(stratification_cols).cost_per_death_averted.describe() / df2.groupby(stratification_cols).gdp_pc.describe()
print('GDPpc multiplier to get ICER')
np.round(t, 1)['mean'].unstack().iloc[1:]